In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import DataLoader, TensorDataset

# 加载数据
df = pd.read_csv('data/student_efficiency_environment.csv')

# 划分特征和标签
X = df[['温度', '湿度', '光亮度']].values
y = df[['学习效率']].values

# 数据划分
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 数据标准化
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 转换为PyTorch张量
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# 创建数据加载器
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [3]:
class MLPEfficiencyModel(torch.nn.Module):
    def __init__(self):
        super(MLPEfficiencyModel, self).__init__()
        self.fc1 = torch.nn.Linear(3, 64)  # 3个输入特征
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(64, 64)
        self.output = torch.nn.Linear(64, 1)  # 输出学习效率

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.output(x)
        return x

# 检查是否有可用的CUDA设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# 将模型移至设备
model = MLPEfficiencyModel().to(device)

Using device: cuda


In [4]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100
for epoch in range(num_epochs):
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)  # 移至设备
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

Epoch 10, Loss: 0.0013644082937389612
Epoch 20, Loss: 0.00023310592223424464
Epoch 30, Loss: 0.0001674455706961453
Epoch 40, Loss: 0.0002007790608331561
Epoch 50, Loss: 0.00015737989451736212
Epoch 60, Loss: 0.00012926968338433653
Epoch 70, Loss: 0.00011794942111009732
Epoch 80, Loss: 0.0001638679241295904
Epoch 90, Loss: 0.00013577647041529417
Epoch 100, Loss: 9.263373794965446e-05


In [5]:
model.eval()  # 进入评估模式
test_loss = 0
with torch.no_grad():  # 测试阶段不需要梯度计算
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()

print(f'Test Loss: {test_loss / len(test_loader)}')

Test Loss: 0.00015759802045067772
